# 🎯 Notebook 9: Object Detection - DETR a YOLO

**Autor:** Praut s.r.o. - AI Integration & Business Automation

V tomto notebooku se naučíme detekovat objekty v obrázcích pomocí moderních modelů DETR (Detection Transformer) a YOLO. Ukážeme si praktické aplikace jako inventarizace, počítání objektů a automatická analýza scén.

## Co se naučíte:
- Detekce objektů s DETR
- Práce s bounding boxes
- Počítání a inventarizace objektů
- Instance segmentation
- Praktické automatizace

In [ ]:
# Instalace potřebných knihoven
!pip install -q transformers accelerate torch torchvision pillow requests matplotlib timm

In [ ]:
import torch
from transformers import DetrImageProcessor, DetrForObjectDetection
from transformers import pipeline
from PIL import Image, ImageDraw, ImageFont
import requests
from io import BytesIO
import matplotlib.pyplot as plt
import matplotlib.patches as patches
import numpy as np
from collections import Counter

# Detekce zařízení
device = "cuda" if torch.cuda.is_available() else "cpu"
print(f"Používám zařízení: {device}")
if device == "cuda":
    print(f"GPU: {torch.cuda.get_device_name(0)}")

## 1. Pomocné funkce

In [ ]:
def load_image(source):
    """Načte obrázek z URL nebo lokální cesty."""
    if source.startswith('http'):
        response = requests.get(source, timeout=10)
        image = Image.open(BytesIO(response.content))
    else:
        image = Image.open(source)
    
    if image.mode != 'RGB':
        image = image.convert('RGB')
    return image

# Barvy pro různé třídy objektů
COLORS = [
    '#FF6B6B', '#4ECDC4', '#45B7D1', '#96CEB4', '#FFEAA7',
    '#DDA0DD', '#98D8C8', '#F7DC6F', '#BB8FCE', '#85C1E9',
    '#F8B500', '#00CED1', '#FF69B4', '#32CD32', '#FF4500'
]

def get_color(idx):
    """Vrátí barvu pro daný index."""
    return COLORS[idx % len(COLORS)]

print("✅ Pomocné funkce připraveny")

## 2. DETR - Detection Transformer

In [ ]:
# Načtení DETR modelu
detr_processor = DetrImageProcessor.from_pretrained("facebook/detr-resnet-50")
detr_model = DetrForObjectDetection.from_pretrained("facebook/detr-resnet-50")

if device == "cuda":
    detr_model = detr_model.to(device)

print("✅ DETR model načten")
print(f"   Počet tříd: {detr_model.config.num_labels}")

In [ ]:
def detect_objects_detr(image, threshold=0.7):
    """
    Detekuje objekty v obrázku pomocí DETR.
    
    Returns:
        Seznam detekovaných objektů s bounding boxy
    """
    # Příprava vstupu
    inputs = detr_processor(images=image, return_tensors="pt")
    if device == "cuda":
        inputs = {k: v.to(device) for k, v in inputs.items()}
    
    # Inference
    with torch.no_grad():
        outputs = detr_model(**inputs)
    
    # Post-processing
    target_sizes = torch.tensor([image.size[::-1]])
    if device == "cuda":
        target_sizes = target_sizes.to(device)
    
    results = detr_processor.post_process_object_detection(
        outputs, 
        target_sizes=target_sizes,
        threshold=threshold
    )[0]
    
    # Formátování výsledků
    detections = []
    for score, label, box in zip(results["scores"], results["labels"], results["boxes"]):
        box = box.cpu().numpy()
        detections.append({
            "label": detr_model.config.id2label[label.item()],
            "score": score.item(),
            "box": {
                "x1": box[0],
                "y1": box[1],
                "x2": box[2],
                "y2": box[3]
            }
        })
    
    return detections

# Test
test_url = "https://upload.wikimedia.org/wikipedia/commons/thumb/4/4d/Cat_November_2010-1a.jpg/1200px-Cat_November_2010-1a.jpg"
image = load_image(test_url)

detections = detect_objects_detr(image, threshold=0.5)
print(f"Nalezeno {len(detections)} objektů:")
for det in detections:
    print(f"  - {det['label']}: {det['score']:.1%}")

In [ ]:
def visualize_detections(image, detections, title="Object Detection"):
    """
    Vizualizuje detekované objekty s bounding boxy.
    """
    fig, ax = plt.subplots(1, figsize=(12, 8))
    ax.imshow(image)
    
    # Unikátní labely pro barvy
    unique_labels = list(set(d['label'] for d in detections))
    label_colors = {label: get_color(i) for i, label in enumerate(unique_labels)}
    
    for det in detections:
        box = det['box']
        label = det['label']
        score = det['score']
        color = label_colors[label]
        
        # Bounding box
        width = box['x2'] - box['x1']
        height = box['y2'] - box['y1']
        
        rect = patches.Rectangle(
            (box['x1'], box['y1']), width, height,
            linewidth=2, edgecolor=color, facecolor='none'
        )
        ax.add_patch(rect)
        
        # Label
        ax.text(
            box['x1'], box['y1'] - 5,
            f"{label}: {score:.1%}",
            color='white', fontsize=10,
            bbox=dict(boxstyle='round', facecolor=color, alpha=0.8)
        )
    
    ax.axis('off')
    ax.set_title(title)
    plt.tight_layout()
    plt.show()

# Vizualizace
visualize_detections(image, detections, "DETR Object Detection")

## 3. Použití Pipeline API (jednodušší přístup)

In [ ]:
# Pipeline pro object detection
detector = pipeline(
    "object-detection",
    model="facebook/detr-resnet-50",
    device=0 if device == "cuda" else -1
)

print("✅ Object detection pipeline připraven")

In [ ]:
# Jednoduchá detekce
results = detector(image, threshold=0.5)

print(f"Detekováno {len(results)} objektů:")
for r in results:
    print(f"  {r['label']}: {r['score']:.1%}")
    print(f"    Box: {r['box']}")

## 4. Počítání objektů a inventarizace

In [ ]:
class ObjectCounter:
    """
    Počítání a inventarizace objektů v obrázcích.
    """
    
    def __init__(self, threshold=0.7):
        self.detector = detector
        self.threshold = threshold
    
    def count(self, image):
        """
        Spočítá objekty podle typu.
        """
        if isinstance(image, str):
            image = load_image(image)
        
        results = self.detector(image, threshold=self.threshold)
        
        # Počítání podle labelů
        counts = Counter(r['label'] for r in results)
        
        return {
            "total": len(results),
            "by_type": dict(counts),
            "detections": results
        }
    
    def inventory(self, images):
        """
        Vytvoří inventář z více obrázků.
        """
        total_inventory = Counter()
        results = []
        
        for i, img_source in enumerate(images):
            try:
                count_result = self.count(img_source)
                total_inventory.update(count_result['by_type'])
                results.append({
                    "image": i,
                    "source": img_source if isinstance(img_source, str) else f"image_{i}",
                    "counts": count_result['by_type'],
                    "total": count_result['total']
                })
            except Exception as e:
                results.append({
                    "image": i,
                    "error": str(e)
                })
        
        return {
            "total_inventory": dict(total_inventory),
            "grand_total": sum(total_inventory.values()),
            "per_image": results
        }

# Test
counter = ObjectCounter(threshold=0.5)
count_result = counter.count(image)

print("Inventarizace obrázku:")
print(f"  Celkem objektů: {count_result['total']}")
print("  Podle typu:")
for obj_type, cnt in count_result['by_type'].items():
    print(f"    - {obj_type}: {cnt}")

In [ ]:
# Test na komplexnějším obrázku
street_url = "https://upload.wikimedia.org/wikipedia/commons/thumb/1/1e/A_traffic_jam.JPG/1280px-A_traffic_jam.JPG"

try:
    street_image = load_image(street_url)
    count_result = counter.count(street_image)
    
    print("\nInventarizace ulice:")
    print(f"  Celkem objektů: {count_result['total']}")
    print("  Podle typu:")
    for obj_type, cnt in sorted(count_result['by_type'].items(), key=lambda x: -x[1]):
        print(f"    - {obj_type}: {cnt}")
    
    visualize_detections(street_image, [
        {'label': d['label'], 'score': d['score'], 'box': d['box']}
        for d in count_result['detections']
    ], "Street Scene Detection")
except Exception as e:
    print(f"Chyba: {e}")

## 5. Analýza scény

In [ ]:
class SceneAnalyzer:
    """
    Komplexní analýza scény z obrázku.
    """
    
    # Kategorie objektů
    CATEGORIES = {
        "vehicles": ["car", "truck", "bus", "motorcycle", "bicycle", "airplane", "train", "boat"],
        "people": ["person"],
        "animals": ["bird", "cat", "dog", "horse", "sheep", "cow", "elephant", "bear", "zebra", "giraffe"],
        "furniture": ["chair", "couch", "bed", "dining table", "toilet"],
        "electronics": ["tv", "laptop", "mouse", "remote", "keyboard", "cell phone"],
        "food": ["banana", "apple", "sandwich", "orange", "broccoli", "carrot", "hot dog", "pizza", "donut", "cake"],
        "kitchen": ["bottle", "wine glass", "cup", "fork", "knife", "spoon", "bowl", "microwave", "oven", "toaster", "sink", "refrigerator"],
        "outdoor": ["traffic light", "fire hydrant", "stop sign", "parking meter", "bench"]
    }
    
    def __init__(self, threshold=0.6):
        self.counter = ObjectCounter(threshold)
    
    def categorize_object(self, label):
        """Přiřadí objekt do kategorie."""
        for category, objects in self.CATEGORIES.items():
            if label in objects:
                return category
        return "other"
    
    def analyze(self, image):
        """
        Provede komplexní analýzu scény.
        """
        if isinstance(image, str):
            image = load_image(image)
        
        # Detekce
        count_result = self.counter.count(image)
        
        # Kategorizace
        category_counts = Counter()
        for det in count_result['detections']:
            cat = self.categorize_object(det['label'])
            category_counts[cat] += 1
        
        # Určení typu scény
        scene_type = self._determine_scene_type(category_counts, count_result['by_type'])
        
        # Prostorová analýza
        spatial = self._spatial_analysis(count_result['detections'], image.size)
        
        return {
            "scene_type": scene_type,
            "total_objects": count_result['total'],
            "objects_by_type": count_result['by_type'],
            "objects_by_category": dict(category_counts),
            "spatial_analysis": spatial,
            "detections": count_result['detections']
        }
    
    def _determine_scene_type(self, categories, objects):
        """Určí typ scény na základě detekovaných objektů."""
        if categories.get('vehicles', 0) > 2:
            return "traffic/street"
        elif categories.get('people', 0) > 3:
            return "crowd/gathering"
        elif categories.get('animals', 0) > 0 and categories.get('outdoor', 0) > 0:
            return "outdoor/nature"
        elif categories.get('furniture', 0) > 1:
            return "indoor/room"
        elif categories.get('kitchen', 0) > 2:
            return "kitchen/dining"
        elif categories.get('electronics', 0) > 1:
            return "office/workspace"
        elif categories.get('food', 0) > 0:
            return "food/dining"
        else:
            return "general"
    
    def _spatial_analysis(self, detections, image_size):
        """Analyzuje prostorové rozložení objektů."""
        if not detections:
            return {"distribution": "empty"}
        
        width, height = image_size
        
        # Středy objektů
        centers = []
        for det in detections:
            box = det['box']
            cx = (box['xmin'] + box['xmax']) / 2 / width
            cy = (box['ymin'] + box['ymax']) / 2 / height
            centers.append((cx, cy))
        
        # Analýza rozložení
        left = sum(1 for c in centers if c[0] < 0.33)
        center = sum(1 for c in centers if 0.33 <= c[0] <= 0.66)
        right = sum(1 for c in centers if c[0] > 0.66)
        
        top = sum(1 for c in centers if c[1] < 0.33)
        middle = sum(1 for c in centers if 0.33 <= c[1] <= 0.66)
        bottom = sum(1 for c in centers if c[1] > 0.66)
        
        return {
            "horizontal": {"left": left, "center": center, "right": right},
            "vertical": {"top": top, "middle": middle, "bottom": bottom},
            "distribution": "centered" if center > left + right else "spread"
        }

# Test
analyzer = SceneAnalyzer(threshold=0.5)

try:
    analysis = analyzer.analyze(street_image)
    
    print("=" * 50)
    print("ANALÝZA SCÉNY")
    print("=" * 50)
    print(f"Typ scény: {analysis['scene_type']}")
    print(f"Celkem objektů: {analysis['total_objects']}")
    print("\nObjekty podle kategorie:")
    for cat, cnt in sorted(analysis['objects_by_category'].items(), key=lambda x: -x[1]):
        print(f"  - {cat}: {cnt}")
except Exception as e:
    print(f"Test na cat image:")
    analysis = analyzer.analyze(image)
    print(f"Typ scény: {analysis['scene_type']}")
    print(f"Objekty: {analysis['objects_by_type']}")

## 6. Detekce specifických objektů

In [ ]:
def find_specific_objects(image, target_labels, threshold=0.5):
    """
    Najde pouze specifické objekty v obrázku.
    
    Args:
        image: Obrázek nebo URL
        target_labels: Seznam hledaných labelů
        threshold: Minimální confidence
    """
    if isinstance(image, str):
        image = load_image(image)
    
    results = detector(image, threshold=threshold)
    
    # Filtrování
    target_labels_lower = [t.lower() for t in target_labels]
    filtered = [
        r for r in results 
        if r['label'].lower() in target_labels_lower
    ]
    
    return {
        "found": len(filtered) > 0,
        "count": len(filtered),
        "detections": filtered
    }

# Test - hledání aut
try:
    result = find_specific_objects(street_image, ["car", "truck", "bus"])
    print(f"Nalezeno vozidel: {result['count']}")
    for det in result['detections'][:5]:
        print(f"  - {det['label']}: {det['score']:.1%}")
except:
    print("Test na základním obrázku:")
    result = find_specific_objects(image, ["cat", "dog"])
    print(f"Nalezeno zvířat: {result['count']}")

## 7. Praktická automatizace: Retail Shelf Analyzer

In [ ]:
class RetailAnalyzer:
    """
    Analyzátor pro retail - počítání produktů, detekce prázdných míst.
    """
    
    PRODUCT_LABELS = ['bottle', 'cup', 'bowl', 'banana', 'apple', 'orange', 
                      'sandwich', 'cake', 'donut', 'pizza']
    
    def __init__(self, threshold=0.5):
        self.threshold = threshold
    
    def analyze_shelf(self, image):
        """
        Analyzuje regál/polici.
        """
        if isinstance(image, str):
            image = load_image(image)
        
        results = detector(image, threshold=self.threshold)
        
        # Počítání produktů
        product_counts = Counter()
        positions = []
        
        for det in results:
            if det['label'] in self.PRODUCT_LABELS:
                product_counts[det['label']] += 1
                
                # Pozice produktu
                box = det['box']
                positions.append({
                    "product": det['label'],
                    "x": (box['xmin'] + box['xmax']) / 2,
                    "y": (box['ymin'] + box['ymax']) / 2
                })
        
        # Analýza pokrytí
        coverage = self._calculate_coverage(results, image.size)
        
        return {
            "total_products": sum(product_counts.values()),
            "product_counts": dict(product_counts),
            "coverage_percent": coverage,
            "needs_restock": coverage < 50,
            "positions": positions
        }
    
    def _calculate_coverage(self, detections, image_size):
        """Vypočítá procento pokrytí plochy."""
        if not detections:
            return 0
        
        width, height = image_size
        total_area = width * height
        
        # Plocha všech bounding boxů
        detected_area = 0
        for det in detections:
            box = det['box']
            box_area = (box['xmax'] - box['xmin']) * (box['ymax'] - box['ymin'])
            detected_area += box_area
        
        return min(100, (detected_area / total_area) * 100)
    
    def compare_shelves(self, image1, image2):
        """
        Porovná dva obrázky regálu (např. před a po).
        """
        analysis1 = self.analyze_shelf(image1)
        analysis2 = self.analyze_shelf(image2)
        
        diff = {}
        all_products = set(analysis1['product_counts'].keys()) | set(analysis2['product_counts'].keys())
        
        for product in all_products:
            count1 = analysis1['product_counts'].get(product, 0)
            count2 = analysis2['product_counts'].get(product, 0)
            diff[product] = count2 - count1
        
        return {
            "before": analysis1,
            "after": analysis2,
            "changes": diff,
            "total_change": analysis2['total_products'] - analysis1['total_products']
        }

# Test
retail = RetailAnalyzer(threshold=0.4)

# Test na dostupném obrázku
result = retail.analyze_shelf(image)
print("Analýza 'regálu':")
print(f"  Produktů: {result['total_products']}")
print(f"  Pokrytí: {result['coverage_percent']:.1f}%")
print(f"  Potřeba doplnit: {'Ano' if result['needs_restock'] else 'Ne'}")

## 8. Bezpečnostní monitoring

In [ ]:
class SecurityMonitor:
    """
    Bezpečnostní monitoring - detekce osob a vozidel.
    """
    
    def __init__(self, threshold=0.6):
        self.threshold = threshold
        self.alert_types = {
            'person': 'Detekována osoba',
            'car': 'Detekováno vozidlo',
            'truck': 'Detekován nákladní vůz',
            'motorcycle': 'Detekována motorka',
            'bicycle': 'Detekováno kolo'
        }
    
    def analyze_frame(self, image, zones=None):
        """
        Analyzuje snímek z kamery.
        
        Args:
            image: Obrázek
            zones: Seznam zón pro monitoring [{"name": "entrance", "box": {...}}]
        """
        if isinstance(image, str):
            image = load_image(image)
        
        results = detector(image, threshold=self.threshold)
        
        # Kategorizace detekcí
        persons = [r for r in results if r['label'] == 'person']
        vehicles = [r for r in results if r['label'] in ['car', 'truck', 'motorcycle', 'bicycle', 'bus']]
        
        # Generování alertů
        alerts = []
        for det in results:
            if det['label'] in self.alert_types:
                alert = {
                    "type": det['label'],
                    "message": self.alert_types[det['label']],
                    "confidence": det['score'],
                    "location": det['box']
                }
                
                # Kontrola zón
                if zones:
                    for zone in zones:
                        if self._in_zone(det['box'], zone['box']):
                            alert['zone'] = zone['name']
                            alert['priority'] = 'high'
                
                alerts.append(alert)
        
        return {
            "timestamp": "2025-01-19T12:00:00",  # V produkci skutečný čas
            "person_count": len(persons),
            "vehicle_count": len(vehicles),
            "alerts": alerts,
            "has_activity": len(results) > 0
        }
    
    def _in_zone(self, detection_box, zone_box):
        """Kontroluje, zda je detekce v dané zóně."""
        det_cx = (detection_box['xmin'] + detection_box['xmax']) / 2
        det_cy = (detection_box['ymin'] + detection_box['ymax']) / 2
        
        return (zone_box['xmin'] <= det_cx <= zone_box['xmax'] and
                zone_box['ymin'] <= det_cy <= zone_box['ymax'])

# Test
security = SecurityMonitor(threshold=0.5)

try:
    monitoring = security.analyze_frame(street_image)
    print("Bezpečnostní monitoring:")
    print(f"  Osob: {monitoring['person_count']}")
    print(f"  Vozidel: {monitoring['vehicle_count']}")
    print(f"  Alertů: {len(monitoring['alerts'])}")
except:
    monitoring = security.analyze_frame(image)
    print(f"Aktivita: {monitoring['has_activity']}")

## 9. Export výsledků

In [ ]:
import json
from datetime import datetime

def export_detections(detections, image_info, output_format="json"):
    """
    Exportuje detekce do různých formátů.
    """
    data = {
        "metadata": {
            "timestamp": datetime.now().isoformat(),
            "image": image_info,
            "total_detections": len(detections)
        },
        "detections": detections
    }
    
    if output_format == "json":
        return json.dumps(data, indent=2, ensure_ascii=False)
    
    elif output_format == "csv":
        lines = ["label,score,xmin,ymin,xmax,ymax"]
        for det in detections:
            box = det['box']
            lines.append(f"{det['label']},{det['score']:.4f},{box['xmin']},{box['ymin']},{box['xmax']},{box['ymax']}")
        return "\n".join(lines)
    
    elif output_format == "yolo":
        # YOLO format: class x_center y_center width height (normalized)
        lines = []
        for det in detections:
            box = det['box']
            # Toto by vyžadovalo normalizaci podle velikosti obrázku
            lines.append(f"{det['label']} {det['score']:.4f}")
        return "\n".join(lines)
    
    return data

# Test
results = detector(image, threshold=0.5)
json_output = export_detections(results, {"source": "test_image", "size": image.size})
print("JSON Export (ukázka):")
print(json_output[:500] + "...")

## 10. Batch zpracování složky

In [ ]:
def process_image_folder(folder_path, threshold=0.5, save_results=True):
    """
    Zpracuje všechny obrázky ve složce.
    """
    from pathlib import Path
    import os
    
    folder = Path(folder_path)
    image_extensions = {'.jpg', '.jpeg', '.png', '.webp', '.bmp'}
    
    results = []
    total_objects = Counter()
    
    # Najdi všechny obrázky
    images = [f for f in folder.iterdir() if f.suffix.lower() in image_extensions]
    
    print(f"Nalezeno {len(images)} obrázků")
    
    for img_path in images:
        try:
            image = load_image(str(img_path))
            detections = detector(image, threshold=threshold)
            
            # Počítání
            counts = Counter(d['label'] for d in detections)
            total_objects.update(counts)
            
            results.append({
                "file": img_path.name,
                "detections": len(detections),
                "objects": dict(counts)
            })
            
            print(f"  ✓ {img_path.name}: {len(detections)} objektů")
            
        except Exception as e:
            results.append({
                "file": img_path.name,
                "error": str(e)
            })
            print(f"  ✗ {img_path.name}: {e}")
    
    summary = {
        "total_images": len(images),
        "processed": len([r for r in results if 'error' not in r]),
        "errors": len([r for r in results if 'error' in r]),
        "total_objects": dict(total_objects),
        "results": results
    }
    
    if save_results:
        output_path = folder / "detection_results.json"
        with open(output_path, 'w', encoding='utf-8') as f:
            json.dump(summary, f, indent=2, ensure_ascii=False)
        print(f"\nVýsledky uloženy do: {output_path}")
    
    return summary

# Ukázka použití (zakomentováno - spustit s vlastní složkou)
# results = process_image_folder("/content/my_images", threshold=0.5)
print("Funkce process_image_folder připravena k použití")

## Shrnutí

V tomto notebooku jsme se naučili:

1. **DETR model** - moderní transformer pro object detection
2. **Bounding boxes** - vizualizace a práce s detekovanými oblastmi
3. **Počítání objektů** - inventarizace a statistiky
4. **Analýza scény** - automatické určení typu prostředí
5. **Retail monitoring** - analýza regálů a produktů
6. **Bezpečnostní monitoring** - detekce osob a vozidel
7. **Export dat** - JSON, CSV a další formáty
8. **Batch zpracování** - hromadné zpracování složek

### Další kroky
- Notebook 10: Image Segmentation a Generování obrázků